In [ ]:
# script to add a new species to the species_list

In [1]:
import pandas as pd
import os
import sys
import copy
sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/thermo/')
sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/kinetics/')
import job
import kineticfun

In [2]:
def species_exists_in_db(species_smiles):
    try:
        job.smiles2index(species_smiles)
        return True
    except IndexError:
        # not found, which means we can add it
        return False

In [3]:
def reaction_exists_in_db(reaction_smiles):
    try:
        index = kineticfun.reaction_smiles2index(reaction_smiles)
        if index < 0:
            return False
        return True
    except IndexError:
        # not found, which means we can add it
        return False

In [4]:
reaction_exists_in_db('hello')

False

In [5]:
reaction_exists_in_db('[H]+[O]_[OH]')

True

In [6]:
# load the base model
import os
import pickle
import datetime
import subprocess
import numpy as np

import rmgpy.chemkin
import rmgpy.data.kinetics
# import cantera as ct


import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# Load the base model and the full 1 week model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model'
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')


3978360192.py:6 <module> INFO Thermo file has default temperature range 300.0 to 1000.0 and 1000.0 to 5000.0


110 species, 1822 reactions


In [8]:
# Load the base model and the full 1 week model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_1week'
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list2, reaction_list2 = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list2)} species, {len(reaction_list2)} reactions')


1099335156.py:6 <module> INFO Thermo file has default temperature range 300.0 to 1000.0 and 1000.0 to 5000.0


130 species, 2488 reactions


In [9]:
for i in range(0, len(reaction_list)):
    match = False
    for j in range(0, len(reaction_list2)):
        if reaction_list[i].is_isomorphic(reaction_list2[j]):
            match = True
            break
    if not match:
        print(reaction_list[i], 'not in mechanism 2')

In [10]:
reactions_to_add = []
for i in range(0, len(reaction_list2)):
    match = False
    for j in range(0, len(reaction_list)):
        if reaction_list2[i].is_isomorphic(reaction_list[j]):
            match = True
            break
    if not match:
        print(str(reaction_list2[i]), '\tnot in mechanism 1')
        reactions_to_add.append(reaction_list2[i])

CH2CO(24) + C4H6(194) <=> C2H(4) + C4H7O(202) 	not in mechanism 1
OH(15) + C3H5-A(94) <=> C[C]CO(9122) 	not in mechanism 1
C[C]CO(9122) <=> H(14) + C3H5O(129) 	not in mechanism 1
O2(2) + O=CC1CCO1(5761) <=> O2(2) + C1=COOCC1(5951) 	not in mechanism 1
O2(2) + O=CC1CCO1(5761) <=> O2(2) + [O]C=CCC[O](5753) 	not in mechanism 1
OH(15) + C4H8-2(189) <=> SC4H8OH(198) 	not in mechanism 1
SC4H9O(217) <=> SC4H8OH(198) 	not in mechanism 1
O2(2) + SC4H8OH(198) <=> C4H8OH-2O2(200) 	not in mechanism 1
C4H8OH-2O2(200) => OH(15) + CH3CHO(35) + CH3CHO(35) 	not in mechanism 1
H2O(8) + C=CC(C)O[O](4243) <=> C4H8OH-2O2(200) 	not in mechanism 1
SC4H8OH(198) <=> O(5) + SC4H9(183) 	not in mechanism 1
SC4H8OH(198) <=> H(14) + [CH2]CC(C)[O](766) 	not in mechanism 1
HO2(16) + O=CC1CCO1(5761) <=> HO2(16) + C1=COOCC1(5951) 	not in mechanism 1
HO2(16) + O=CC1CCO1(5761) <=> HO2(16) + [O]C=CCC[O](5753) 	not in mechanism 1
CH2CHO(21) + C2H4O1-2(51) <=> CH2CHO(21) + [CH2]C[O](703) 	not in mechanism 1
OH(15) + C3H6(12)

H(14) + CH2OH(34) <=> OH(15) + CH3(18) 	not in mechanism 1
O(5) + CH2OH(34) <=> OH(15) + CH2O(9) 	not in mechanism 1
OH(15) + CH2OH(34) <=> H2O(8) + CH2O(9) 	not in mechanism 1
O2(2) + CH2OH(34) <=> HO2(16) + CH2O(9) 	not in mechanism 1
CH3(18) + CH2OH(34) <=> CH2O(9) + CH4(10) 	not in mechanism 1
H(14) + CH3OH(30) <=> H2(13) + CH2OH(34) 	not in mechanism 1
O(5) + CH3OH(30) <=> OH(15) + CH2OH(34) 	not in mechanism 1
OH(15) + CH3OH(30) <=> H2O(8) + CH2OH(34) 	not in mechanism 1
O2(2) + CH3OH(30) <=> HO2(16) + CH2OH(34) 	not in mechanism 1
HO2(16) + CH3OH(30) <=> H2O2(17) + CH2OH(34) 	not in mechanism 1
CH2(23) + CH3OH(30) <=> CH3(18) + CH2OH(34) 	not in mechanism 1
CH3(18) + CH3OH(30) <=> CH2OH(34) + CH4(10) 	not in mechanism 1
CH3OH(30) + C2H(4) <=> CH2OH(34) + C2H2(25) 	not in mechanism 1
CH3OH(30) + C2H3(22) <=> CH2OH(34) + C2H4(11) 	not in mechanism 1
OH(15) + CH2CO(24) <=> CO(6) + CH2OH(34) 	not in mechanism 1
OH(15) + CH2CHO(21) <=> HCO(19) + CH2OH(34) 	not in mechanism 1
CH3OH(30

CH3O2H(46) + C3H5O(129) <=> CH3O2(45) + C3H5OH(133) 	not in mechanism 1
C3H5O(129) + SC4H9(183) <=> C3H5OH(133) + C4H8-2(189) 	not in mechanism 1
C3H5O(129) + PC4H9(182) <=> C3H5OH(133) + C4H8-1(188) 	not in mechanism 1
C3H5O(129) + SC4H9(183) <=> C3H5OH(133) + C4H8-1(188) 	not in mechanism 1
C2H5O(49) + C3H5O(129) <=> CH3CHO(35) + C3H5OH(133) 	not in mechanism 1
C3H5O(129) + C3H6(12) <=> C3H5-A(94) + C3H5OH(133) 	not in mechanism 1
C3H5O(129) + [CH2]C[CH]C(748) <=> C3H5OH(133) + C4H71-3(190) 	not in mechanism 1
C3H5O(129) + C4H8-1(188) <=> C3H5OH(133) + C4H71-3(190) 	not in mechanism 1
C3H5O(129) + C4H8-2(189) <=> C3H5OH(133) + C4H71-3(190) 	not in mechanism 1
C3H5O(129) + C4H71-4(191) <=> C3H5OH(133) + C4H6(194) 	not in mechanism 1
C3H5O(129) + C4H71-3(190) <=> C3H5OH(133) + C4H6(194) 	not in mechanism 1
C3H5O(129) + C3H5O(129) <=> C2H3CHO(74) + C3H5OH(133) 	not in mechanism 1
C3H5OH(133) <=> H(14) + C3H5O(129) 	not in mechanism 1
C3H5OH(133) <=> CH2OH(34) + C2H3(22) 	not in mechanis

C2H6(32) + C4H8OOH2-4(223) <=> C2H5(33) + SC4H9O2H(187) 	not in mechanism 1
O2CH2CHO(56) + OC1CO1(5161) <=> O2CH2CHO(56) + [O]C[CH]O(993) 	not in mechanism 1
OH(15) + OC1CO1(5161) <=> OH(15) + [O]C[CH]O(993) 	not in mechanism 1
CH2CHO(21) + OC1CO1(5161) <=> CH2CHO(21) + [O]C[CH]O(993) 	not in mechanism 1
CO2(7) + C4H8-1(188) <=> CO2(7) + C4H8-2(189) 	not in mechanism 1
CO2(7) + C4H8-1(188) <=> CH2CO(24) + SC3H5OH(135) 	not in mechanism 1
OH(15) + C2H4(11) <=> PC2H4OH(59) 	not in mechanism 1
H(14) + PC2H4OH(59) <=> H2O(8) + C2H4(11) 	not in mechanism 1
H(14) + PC2H4OH(59) <=> CH3(18) + CH2OH(34) 	not in mechanism 1
H(14) + PC2H4OH(59) <=> OH(15) + C2H5(33) 	not in mechanism 1
O(5) + PC2H4OH(59) <=> CH2O(9) + CH2OH(34) 	not in mechanism 1
HO2(16) + PC2H4OH(59) => OH(15) + CH2O(9) + CH2OH(34) 	not in mechanism 1
O2(2) + PC2H4OH(59) <=> OH(15) + CH2O(9) + CH2O(9) 	not in mechanism 1
C2H5O(49) <=> PC2H4OH(59) 	not in mechanism 1
OH(15) + C2H3CHO(74) <=> CO(6) + PC2H4OH(59) 	not in mechanism

CH2CHO(21) + [CH2]C[CH]C(748) <=> C2H3OH(58) + C4H71-3(190) 	not in mechanism 1
CH2CHO(21) + C4H71-4(191) <=> C2H3OH(58) + C4H6(194) 	not in mechanism 1
CH2CHO(21) + C4H71-3(190) <=> C2H3OH(58) + C4H6(194) 	not in mechanism 1
CH2CHO(21) + C3H5O(129) <=> C2H3OH(58) + C2H3CHO(74) 	not in mechanism 1
OCHO(38) + PC2H4OH(59) <=> HOCHO(40) + C2H3OH(58) 	not in mechanism 1
CH3O(31) + PC2H4OH(59) <=> CH3OH(30) + C2H3OH(58) 	not in mechanism 1
CH2OH(34) + PC2H4OH(59) <=> CH3OH(30) + C2H3OH(58) 	not in mechanism 1
PC2H4OH(59) + C3H5O(129) <=> C2H3OH(58) + C3H5OH(133) 	not in mechanism 1
CH2CHO(21) + PC2H4OH(59) <=> C2H3OH(58) + C2H3OH(58) 	not in mechanism 1
O(5) + C2H3OH(58) <=> [O]C[CH]O(993) 	not in mechanism 1
C2H3OH(58) <=> OH(15) + C2H3(22) 	not in mechanism 1
C2H3OH(58) <=> H(14) + CH2CHO(21) 	not in mechanism 1
C2H3OH(58) <=> CH2(23) + CH2O(9) 	not in mechanism 1
C2H3OH(58) <=> H(14) + CH3CO(20) 	not in mechanism 1
HO2(16) + C2H3OH(58) <=> OH(15) + [O]C[CH]O(993) 	not in mechanism 1
C2H4

CH2O(9) + C2H3CO(75) <=> HCO(19) + C2H3CHO(74) 	not in mechanism 1
C2H3CO(75) + [CH2]C[CH]C(748) <=> C2H3CHO(74) + C4H71-4(191) 	not in mechanism 1
C2H3CHO(74) + C4H71-4(191) <=> C2H3CO(75) + C4H8-1(188) 	not in mechanism 1
CH3CHO(35) + C2H3CO(75) <=> CH3CO(20) + C2H3CHO(74) 	not in mechanism 1
CO2(7) + C2H3CHO(74) <=> OCHO(38) + C2H3CO(75) 	not in mechanism 1
C2H3CHO(74) + C2H3CHO(74) <=> C2H3CO(75) + C3H5O(129) 	not in mechanism 1
[CH2]C(O)C=O(5659) <=> H2O(8) + C2H3CO(75) 	not in mechanism 1
CO(6) + PC2H4OH(59) <=> H2O(8) + C2H3CO(75) 	not in mechanism 1
PC2H4OH(59) + C2H3CO(75) <=> C2H3OH(58) + C2H3CHO(74) 	not in mechanism 1
HCO(19) + C2H3OH(58) <=> H2O(8) + C2H3CO(75) 	not in mechanism 1
HCCO(27) + C2H3CHO(74) <=> CH2CO(24) + C2H3CO(75) 	not in mechanism 1
C2H3CHO(74) + C4H71-1(192) <=> C2H3CO(75) + C4H8-1(188) 	not in mechanism 1
OCHO(38) + C2H3CHO(74) <=> HOCHO(40) + C2H3CO(75) 	not in mechanism 1
CH2OH(34) + C2H3CHO(74) <=> CH3OH(30) + C2H3CO(75) 	not in mechanism 1
C2H3CO(75)

In [11]:
len(reactions_to_add)

659

In [12]:
# Read the species csv
DFT_DIR = '/work/westgroup/harris.se/autoscience/autoscience/butane/dft'
species_csv = os.path.join(DFT_DIR, 'species_list.csv')
species_df = pd.read_csv(species_csv)

# Read the reaction csv
# DFT_DIR = '/work/westgroup/harris.se/autoscience/autoscience/butane/dft'
reaction_csv = os.path.join(DFT_DIR, 'reaction_list.csv')
reaction_df = pd.read_csv(reaction_csv)

if 'Unnamed: 0' in reaction_df.keys():  
    reaction_df = reaction_df.drop(columns=['Unnamed: 0'])

print(reaction_df)

         i                                               name  \
0        0                    H(14) + O2(2) <=> OH(15) + O(5)   
1        1                   H2(13) + O(5) <=> H(14) + OH(15)   
2        2                 H2(13) + OH(15) <=> H2O(8) + H(14)   
3        3                  OH(15) + OH(15) <=> H2O(8) + O(5)   
4        4                           H2(13) <=> H(14) + H(14)   
...    ...                                                ...   
2466  2466     [CH2]C(=O)COO(911) <=> O=O(694) + CH3COCH2(70)   
2467  2467     CO(6) + C2H5O2(47) <=> O=O(694) + CH3COCH2(70)   
2468  2468  OH(15) + O=C1COC1(917) <=> O=O(694) + CH3COCH2...   
2469  2469       HO2(16) + C3H5-A(94) <=> O=O(694) + C3H6(12)   
2470  2470                                 O=O(694) <=> O2(2)   

                                SMILES  
0                  [H]+[O][O]_[OH]+[O]  
1                  [H][H]+[O]_[H]+[OH]  
2                    [H][H]+[OH]_O+[H]  
3                      [OH]+[OH]_O+[O]  
4             

In [13]:
reaction_df = reaction_df[0:len(reaction_list)]

In [14]:
reaction_df

,i,name,SMILES
0,0,H(14) + O2(2) <=> OH(15) + O(5),[H]+[O][O]_[OH]+[O]
1,1,H2(13) + O(5) <=> H(14) + OH(15),[H][H]+[O]_[H]+[OH]
2,2,H2(13) + OH(15) <=> H2O(8) + H(14),[H][H]+[OH]_O+[H]
3,3,OH(15) + OH(15) <=> H2O(8) + O(5),[OH]+[OH]_O+[O]
4,4,H2(13) <=> H(14) + H(14),[H][H]_[H]+[H]
...,...,...,...
1817,1817,[O]C=CCC[O](5753) <=> C2H3CHO(74) + CH2O(9),[O]C=CCC[O]_C=CC=O+C=O
1818,1818,[O]C=CCC[O](5753) <=> C1=COOCC1(5951),[O]C=CCC[O]_C1=COOCC1
1819,1819,O=CC1CCO1(5761) <=> C2H3CHO(74) + CH2O(9),O=CC1CCO1_C=CC=O+C=O
1820,1820,C1=COOCC1(5951) <=> O=CC1CCO1(5761),C1=COOCC1_O=CC1CCO1


In [15]:
# add all the new reactions:
starting_length = len(reaction_df)
i = 0
for j in range(0, len(reactions_to_add)):
    try:
        smiles = reactions_to_add[j].smiles
    except AttributeError:
        smiles = kineticfun.reaction2smiles(reactions_to_add[j])
    if smiles in reaction_df['SMILES'].values:
        print(f'skipping {reactions_to_add[j]} because it was already added')
        continue
#     if reaction_exists_in_db(kineticfun.reaction2smiles(reactions_to_add[i])):
#         print(f'skipping {reactions_to_add[i]} because it was already added')
#         continue
    new_entry = {
        'i': starting_length + i,
        'name': str(reactions_to_add[j]),
        'SMILES': kineticfun.reaction2smiles(reactions_to_add[j])
    }
#     print(f'adding new entry {new_entry}')
    reaction_df = reaction_df.append(new_entry, ignore_index=True)
    i = i + 1

skipping CH2CO(24) + C2H3CHO(74) <=> HCCO(27) + C3H5O(129) because it was already added
skipping CH2CO(24) + C4H6(194) <=> HCCO(27) + C4H71-4(191) because it was already added
skipping CH2CO(24) + C4H6(194) <=> HCCO(27) + C4H71-3(190) because it was already added
skipping CH2CO(24) + CH2CO(24) <=> HCCO(27) + CH2CHO(21) because it was already added
skipping CO2(7) + C4H8-1(188) <=> CO2(7) + C4H8-2(189) because it was already added
skipping CH2O(9) + C2H3OH(58) <=> CH2(23) + [O]C[CH]O(993) because it was already added
skipping CH3O(31) + OC1CO1(5161) <=> CH3O2(45) + C2H3OH(58) because it was already added
skipping C2H3CHO(74) + C2H3CHO(74) <=> C2H3CO(75) + C3H5O(129) because it was already added
skipping C2H3CHO(74) + C2H3CHO(74) <=> C2H3CO(75) + C3H5O(129) because it was already added
skipping CO2(7) + C2H3OH(58) <=> CO2(7) + CH3CHO(35) because it was already added


In [16]:
reaction_df

,i,name,SMILES
0,0,H(14) + O2(2) <=> OH(15) + O(5),[H]+[O][O]_[OH]+[O]
1,1,H2(13) + O(5) <=> H(14) + OH(15),[H][H]+[O]_[H]+[OH]
2,2,H2(13) + OH(15) <=> H2O(8) + H(14),[H][H]+[OH]_O+[H]
3,3,OH(15) + OH(15) <=> H2O(8) + O(5),[OH]+[OH]_O+[O]
4,4,H2(13) <=> H(14) + H(14),[H][H]_[H]+[H]
...,...,...,...
2466,2466,[CH2]C(=O)COO(911) <=> O=O(694) + CH3COCH2(70),[CH2]C(=O)COO_O=O+[CH2]C(C)=O
2467,2467,CO(6) + C2H5O2(47) <=> O=O(694) + CH3COCH2(70),[C-]#[O+]+CCO[O]_O=O+[CH2]C(C)=O
2468,2468,OH(15) + O=C1COC1(917) <=> O=O(694) + CH3COCH2...,[OH]+O=C1COC1_O=O+[CH2]C(C)=O
2469,2469,HO2(16) + C3H5-A(94) <=> O=O(694) + C3H6(12),[O]O+[CH2]C=C_O=O+C=CC


In [17]:
# save the new results
reaction_df.to_csv(reaction_csv)

In [18]:
# add a specific reaction
import rmgpy.reaction
import rmgpy.species

import rmgpy

In [19]:
rmgpy.settings

{'database.directory': '/home/harris.se/rmg/RMG-database/input',
 'test_data.directory': '/home/harris.se/rmg/RMG-Py/rmgpy/test_data'}

In [20]:
# load the species dictionary

family = 'Disproportionation'
spec_dict = rmgpy.chemkin.load_species_dictionary(os.path.join(rmgpy.settings['database.directory'], 'kinetics', 'families', family, 'training', 'dictionary.txt'))



In [21]:
# load the Disproportionation training depo
ref_library_path = os.path.join(rmgpy.settings['database.directory'], 'kinetics')
ref_database = rmgpy.data.kinetics.KineticsDatabase()
ref_database.load(
    ref_library_path,
    libraries=[],
    families=[family]
)
training_depo = ref_database.families['Disproportionation'].get_training_depository()
print(len(training_depo.entries), 'training reactions')

137 training reactions


In [22]:
list_of_df_reactions = []
for i in range(0, len(reaction_df)):
    list_of_df_reactions.append(kineticfun.smiles2reaction(reaction_df['SMILES'].values[i]))

In [23]:
# def rxn_in_df(rxn):
#     for i in range(0, len(reaction_df)):
#         ref_rxn = kineticfun.smiles2reaction(reaction_df['SMILES'].values[i])
#         if rxn.is_isomorphic(ref_rxn):
#             return True
#     return False


def rxn_in_df(rxn):
    for i in range(0, len(list_of_df_reactions)):
        ref_rxn = list_of_df_reactions[i]
        if rxn.is_isomorphic(ref_rxn):
            return True
    return False

In [24]:
# add everything in the Disproportionation family training depo

for key in training_depo.entries:
    new_reaction = training_depo.entries[key].item
    
    
    # check if it's already in reaction_df
    in_df = rxn_in_df(new_reaction)
    if in_df:  # need to repopulate the list
        continue
        
    

    # we need to substitute the species label for existing ones
    for sp in new_reaction.reactants + new_reaction.products:
        
        index = np.where(species_df['SMILES'] == sp.smiles)[0][0]
        
        if index >= 0 and str(sp) != species_df['name'].values[index]:
            sp.label = species_df['name'].values[index]

    print(new_reaction)
    new_entry = {
        'i': len(reaction_df),
        'name': str(new_reaction),
        'SMILES': kineticfun.reaction2smiles(new_reaction)
    }
#     print(f'adding new entry {new_entry}')
    reaction_df = reaction_df.append(new_entry, ignore_index=True)


C2H5(33) + IC3H7(93) <=> C3H8 + C2H4(11)
C2H5(33) + C4H9 <=> C4H10 + C2H4(11)
C2H3S + NC3H7(92) <=> C2H4S + C3H6(12)
C4H71-3(190) + C2H5S <=> C4H8-1(188) + C2H4S-2
C5H7 + C2H5S <=> C5H8 + C2H4S-2
C2H3S + C4H9-2 <=> C2H4S + C4H8-2
C4H9-2 + CH3S <=> CH4S + C4H8-2
NC3H7(92) + IC3H7(93) <=> C3H8 + C3H6(12)
NC3H7(92) + C4H9 <=> C4H10 + C3H6(12)
HS2 + C5H9 <=> H2S2 + C5H8-2
HS2 + CH3S-2 <=> H2S2 + CH2S
C4H9-2 + O2(2) <=> HO2(16) + C4H8-2
C2H(4) + C4H9-2 <=> C2H2(25) + C4H8-2
H(14) + C4H9-2 <=> H2(13) + C4H8-2
CH3(18) + C4H9-2 <=> CH4(10) + C4H8-2
C4H9-2 + C2H5(33) <=> C2H6(32) + C4H8-2
CH2OH(34) + C4H9-2 <=> CH3OH(30) + C4H8-2
C3H5-A(94) + C4H9-2 <=> C3H6(12) + C4H8-2
C4H9-2 + IC3H7(93) <=> C3H8 + C4H8-2
C4H9-2 + C4H9 <=> C4H10 + C4H8-2
C2H3(22) + C4H9-2 <=> C2H4(11) + C4H8-2
OH(15) + C4H9-2 <=> H2O(8) + C4H8-2
C3H5-A(94) + O2(2) <=> HO2(16) + C3H4
CH3(18) + C3H5-A(94) <=> CH4(10) + C3H4
C3H5-A(94) + C2H5(33) <=> C2H6(32) + C3H4
C3H5-A(94) + C3H5-A(94) <=> C3H6(12) + C3H4
C3H5-A(94) + IC3H7(

In [25]:
reaction_df

,i,name,SMILES
0,0,H(14) + O2(2) <=> OH(15) + O(5),[H]+[O][O]_[OH]+[O]
1,1,H2(13) + O(5) <=> H(14) + OH(15),[H][H]+[O]_[H]+[OH]
2,2,H2(13) + OH(15) <=> H2O(8) + H(14),[H][H]+[OH]_O+[H]
3,3,OH(15) + OH(15) <=> H2O(8) + O(5),[OH]+[OH]_O+[O]
4,4,H2(13) <=> H(14) + H(14),[H][H]_[H]+[H]
...,...,...,...
2563,2563,OH(15) + CH2NO <=> H2O(8) + CHNO,[OH]+[O-][N+]=C_O+[O-][N+]#C
2564,2564,CH3(18) + CH2NO <=> CH4(10) + CHNO,[CH3]+[O-][N+]=C_C+[O-][N+]#C
2565,2565,H2N + CH2NO <=> H3N + CHNO,[NH2]+[O-][N+]=C_N+[O-][N+]#C
2566,2566,C5H7 + C4H71-3(190) <=> C5H8 + C4H6(194),C=C[CH]C=C+C=C[CH]C_C=CCC=C+C=CC=C


In [26]:
# save the new results
reaction_df.to_csv(reaction_csv)

In [ ]:
my_reaction = 'C4H7-2 + O2 <=> HO2 + C4H6'
my_reaction = 'C5H9-2 + O2 <=> HO2 + C5H8-3'
for key in training_depo.entries:
    # get the index in the database:

    if str(training_depo.entries[key]) == my_reaction:
#         print(training_depo.entries[key].index, training_depo.entries[key])
        index = kineticfun.reaction_smiles2index(kineticfun.reaction2smiles(training_depo.entries[key].item))
        print(training_depo.entries[key].index, index, training_depo.entries[key])
        
        for sp in training_depo.entries[key].item.reactants + training_depo.entries[key].item.products:
            print(sp.smiles)

In [ ]:
spec_dict['C6H5']

In [ ]:
# 136, rank 3
rxn = rmgpy.reaction.Reaction()
rxn.reactants = [spec_dict['C6H5'], spec_dict['C6H5-2']]
rxn.products = [spec_dict['C6H6'], spec_dict['C6H4']]


In [ ]:
# check if the species is done calculating

for sp in rxn.reactants + rxn.products:
    index = job.smiles2index(sp.smiles)
    print(index, sp.smiles)


In [ ]:
# check if it's already in there
smiles = kineticfun.reaction2smiles(rxn)
already_in_db = False
if smiles in reaction_df['SMILES'].values:
    print(f'skipping {rxn} because it was already added')
    already_in_db = True
if reaction_exists_in_db(smiles):
    print(f'skipping {rxn} because it was already added')
    already_in_db = True

In [ ]:
# add single new reaction:
if not already_in_db:
    new_entry = {
        'i': len(reaction_df),
        'name': str(rxn),
        'SMILES': smiles
    }
    reaction_df = reaction_df.append(new_entry, ignore_index=True)


In [ ]:
reaction_df

In [ ]:
# save the new results
reaction_df.to_csv(reaction_csv)